## Linear Classifier in TensorFlow 
Using Low Level API in Eager Execution mode

### Load tensorflow

In [1]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [2]:
import tensorflow as tf
print(tf.__version__)

2.0.0


In [0]:
#Enable Eager Execution if using tensflow version < 2.0
#From tensorflow v2.0 onwards, Eager Execution will be enabled by default
tf.enable_eager_execution()

### Collect Data

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [0]:
import pandas as pd

In [0]:
data = pd.read_csv('/content/drive/My Drive/great-learning/neural networks/prices.csv')

### Check all columns in the dataset

In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 851264 entries, 0 to 851263
Data columns (total 7 columns):
date      851264 non-null object
symbol    851264 non-null object
open      851264 non-null float64
close     851264 non-null float64
low       851264 non-null float64
high      851264 non-null float64
volume    851264 non-null float64
dtypes: float64(5), object(2)
memory usage: 45.5+ MB


### Drop columns `date` and  `symbol`

In [0]:
data.drop(columns=['date','symbol'],inplace=True)

In [15]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set
Target 'Volume' has very high values. Divide 'Volume' by 1000,000

In [16]:
data = data.iloc[0:1000,:]
data.shape

(1000, 5)

In [17]:
data['volume'] = data['volume']/1000000
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2.1636
1,125.239998,119.980003,119.940002,125.540001,2.3864
2,116.379997,114.949997,114.930000,119.739998,2.4895
3,115.480003,116.620003,113.500000,117.440002,2.0063
4,117.010002,114.970001,114.089996,117.330002,1.4086


### Divide the data into train and test sets

In [0]:
from sklearn.model_selection import train_test_split
X = data.drop(columns=['volume'])
Y = data['volume']
train_x,test_x,train_y,test_y=train_test_split(X,Y,test_size=.30,random_state=2)

#### Convert Training and Test Data to numpy float32 arrays


In [0]:
import numpy as np
# fix random seed for reproducibility
np.random.seed(7)

In [0]:
train_x =np.array(train_x).astype('float32')
test_x = np.array(test_x).astype('float32')
train_y =np.array(train_y).astype('float32')
test_y = np.array(test_y).astype('float32')

In [21]:
train_x.shape

(700, 4)

### Normalize the data
You can use Normalizer from sklearn.preprocessing

In [0]:
from sklearn.preprocessing import Normalizer

transformer = Normalizer()
train_x = transformer.fit_transform(train_x)
test_x = transformer.transform(test_x)

## Building the Model in tensorflow

1.Define Weights and Bias, use tf.zeros to initialize weights and Bias

In [0]:
#We are initializing weights and Bias of (Input-Hidden Layer)
#3 neurons in the hidden layer
W1 = tf.zeros(shape=(4, 3))
b1 = tf.zeros(shape=(3))

In [0]:
#We are initializing weights and Bias of (Hidden-Output)
W2 = tf.zeros(shape=(3, 1))
b2 = tf.zeros(shape=(1))

2.Define a function to calculate prediction

In [0]:
@tf.function
def prediction(features, w1, b1, w2, b2):
    net1 = tf.add(tf.matmul(features, w1), b1)
    y = tf.sigmoid(net1)
    y_pred = tf.matmul(y, w2)+b2
    return y_pred

3.Loss (Cost) Function [Mean square error]

In [0]:
@tf.function
def loss(y_actual, y_predicted):
    diff = y_actual - y_predicted
    sqr = tf.square(diff)
    avg = tf.reduce_mean(sqr)
    return avg

4.Function to train the Model

1.   Record all the mathematical steps to calculate Loss
2.   Calculate Gradients of Loss w.r.t weights and bias
3.   Update Weights and Bias based on gradients and learning rate to minimize loss

In [0]:
@tf.function
def train(x, y_actual, w1, b1, w2, b2, learning_rate=0.01):
    
    # Record mathematical operations on 'tape' to calculate loss
    with tf.GradientTape() as t:
        t.watch([w1,b1,w2,b2])
        current_prediction = prediction(x, w1, b1, w2, b2)
        current_loss = loss(y_actual, current_prediction)
    
    #Calculate Gradients for Loss with respect to Weights and Bias
    dw1,db1,dw2,db2 = t.gradient(current_loss,[w1, b1,w2,b2])
    
    #Update Weights at output layer
    w2 = w2 - learning_rate*dw2
    b2 = b2 - learning_rate*db2
    
     #Update Weights at hidden layer
    w1 = w1 - learning_rate*dw1
    b1 = b1 - learning_rate*db1
    
    return w1,b1,w2,b2

## Train the model for 100 epochs 
1. Observe the training loss at every iteration
2. Observe Train loss at every 5th iteration

In [28]:
for i in range(100):    
    W1, b1, W2, b2 = train(train_x, train_y, W1, b1, W2, b2)
    if (i%5 == 0):
      print('Current Training Loss on iteration', i, loss(train_y, prediction(train_x, W1, b1, W2, b2)).numpy())

Current Training Loss on iteration 0 248.37215
Current Training Loss on iteration 5 239.36063
Current Training Loss on iteration 10 232.78564
Current Training Loss on iteration 15 227.99687
Current Training Loss on iteration 20 224.61003
Current Training Loss on iteration 25 222.31659
Current Training Loss on iteration 30 220.83464
Current Training Loss on iteration 35 219.91666
Current Training Loss on iteration 40 219.3682
Current Training Loss on iteration 45 219.04948
Current Training Loss on iteration 50 218.86838
Current Training Loss on iteration 55 218.76695
Current Training Loss on iteration 60 218.71063
Current Training Loss on iteration 65 218.67987
Current Training Loss on iteration 70 218.66318
Current Training Loss on iteration 75 218.65414
Current Training Loss on iteration 80 218.6491
Current Training Loss on iteration 85 218.6465
Current Training Loss on iteration 90 218.64522
Current Training Loss on iteration 95 218.64438


### Get the shapes and values of W and b

In [29]:
W1.shape

TensorShape([4, 3])

In [30]:
W1.numpy()

array([[0.20475148, 0.20475148, 0.20475148],
       [0.20546015, 0.20546015, 0.20546015],
       [0.20263866, 0.20263866, 0.20263866],
       [0.20724544, 0.20724544, 0.20724544]], dtype=float32)

In [31]:
W2.shape

TensorShape([3, 1])

In [32]:
W2.numpy()

array([[1.4592286],
       [1.4592286],
       [1.4592286]], dtype=float32)

In [33]:
b1.shape

TensorShape([3])

In [34]:
b1.numpy()

array([0.41007116, 0.41007116, 0.41007116], dtype=float32)

In [35]:
b2.shape

TensorShape([1])

In [36]:
b2.numpy()

array([2.5876803], dtype=float32)

### Model Prediction on 1st Examples in Test Dataset

In [37]:
record1 = np.reshape(test_x[0,:], (1,4)) 
print(prediction(record1, W1, b1, W2, b2).numpy())

[[5.6269484]]


In [38]:
print(loss(test_y[0],prediction(record1, W1, b1, W2, b2)).numpy())

21.672268


## Classification using tf.Keras

In this exercise, we will build a Deep Neural Network using tf.Keras. We will use Iris Dataset for this exercise.

### Load the given Iris data using pandas (Iris.csv)

In [0]:
iris = pd.read_csv('/content/drive/My Drive/great-learning/neural networks/11_Iris.csv')

In [41]:
iris.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [0]:
iris_encoded = pd.get_dummies(iris)

In [67]:
iris_encoded.shape

(150, 8)

### Splitting the data into feature set and target set

In [0]:
X = iris_encoded[['SepalLengthCm','SepalWidthCm','PetalLengthCm','PetalWidthCm']]
y = iris_encoded[['Species_Iris-setosa','Species_Iris-versicolor','Species_Iris-virginica']]

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [0]:
X = X.values
y = y.values
X_train = X_train.values
X_test = X_test.values
y_train = y_train.values
y_test = y_test.values

In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense 
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras import regularizers
from tensorflow.keras import optimizers
from tensorflow.keras.backend import backend

###  Building Model in tf.keras

Build a Linear Classifier model  <br>
1.  Use Dense Layer  with input shape of 4 (according to the feature set) and number of outputs set to 3<br> 
2. Apply Softmax on Dense Layer outputs <br>
3. Use SGD as Optimizer
4. Use categorical_crossentropy as loss function 

In [0]:
# define base model
def DL_model():
    model = Sequential()
    model.add(BatchNormalization())
    model.add(Dense(3, input_dim=4, kernel_initializer='normal', activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
    return model

### Model Training 

In [128]:
model = DL_model()
model.fit(X_train,y_train,epochs=5000, batch_size=X.shape[0],verbose=0)

### Model Prediction

In [141]:
probas = model.predict(X_test)
class_result=np.argmax(probas,axis=-1)
class_result
#species = ['Iris-setosa','Iris-versicolor','Iris-virginica']

array([2, 0, 2, 2, 2, 1, 2, 1, 0, 2, 2, 1, 2, 1, 1, 2, 0, 0, 2, 1, 1, 2,
       2, 2, 1, 2, 0, 1, 0, 0])

In [131]:
score = model.evaluate(X_test,y_test,verbose=0,batch_size=X.shape[0])
print(score)
model.metrics_names

[0.21873445808887482, 0.9]


['loss', 'accuracy']

### Save the Model

In [132]:
model.save

<bound method Network.save of <tensorflow.python.keras.engine.sequential.Sequential object at 0x7f9cc84c4b70>>

### Build and Train a Deep Neural network with 2 hidden layer  - Optional - For Practice

Does it perform better than Linear Classifier? What could be the reason for difference in performance?

In [0]:
def DL_model2(X_train, y_train, iterations, learning_rate, Lambda, verb=True):

    ## hyperparameters
    iterations = iterations
    learning_rate = learning_rate
    hidden_nodes = 256
    output_nodes = 3

    model = Sequential()
    model.add(Dense(hidden_nodes, input_dim=4, kernel_initializer='normal', activation='sigmoid'))
    model.add(Dense(hidden_nodes, activation='sigmoid'))
    model.add(Dense(output_nodes, activation='softmax', kernel_regularizer=regularizers.l2(Lambda)))
    
    sgd = optimizers.SGD(lr=learning_rate, decay=1e-6, momentum=0.9)
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
    
    # Fit the model
    model.fit(X_train, y_train, epochs=iterations, batch_size=X_train.shape[0], verbose= 0)
    #score = model.evaluate(X_train, y_train, verbose=0)
    
    return model

In [134]:
model = DL_model2(X_train,y_train,5000,0.01,0)
score = model.evaluate(X_test,y_test,verbose=0)
print(score)
print(model.metrics_names)

[0.07089259475469589, 0.96666664]
['loss', 'accuracy']


After adding 2 hidden layers, with 256 neurons in each layer, the accuracy shoots up from 90% to 96.67%